In [2]:
import coin_utils_delta as coin_utils
from spark_connector.session_utils import get_spark_session
import pandas as pd
from pyspark.sql.functions import col

In [44]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType, TimestampType, LongType, MapType
from pyspark.sql.functions import col, to_timestamp, lit


In [3]:
session = get_spark_session()

24/09/08 23:54:31 WARN Utils: Your hostname, Sandeshs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.149.131.208 instead (on interface en0)
24/09/08 23:54:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/sandeshmemane/.ivy2/cache
The jars for the packages stored in: /Users/sandeshmemane/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c87eb7f2-960d-41e7-b054-7b733500c519;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/sandeshmemane/opt/anaconda3/envs/env-coin-de/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found org.apache.hadoop#hadoop-aws;3.4.0 in central
	found software.amazon.awssdk#bundle;2.23.19 in central
	found org.wildfly.openssl#wildfly-openssl;1.1.3.Final in central
:: resolution report :: resolve 294ms :: artifacts dl 12ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.4.0 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.1.3.Final from central in [default]
	software.amazon.awssdk#bundle;2.23.19 from central in [default]
	---------------------------------------------------------------------
	|                  |    

In [4]:
check = coin_utils.load_coin_metadata(session)

2024-09-08 23:55:42,275 - INFO - Coins fetched: 71939
2024-09-08 23:55:42,289 - INFO - Coins first 5 rows:                  id          name symbol  rank  is_new  is_active   type
0       btc-bitcoin       Bitcoin    BTC     1   False       True   coin
1      eth-ethereum      Ethereum    ETH     2   False       True   coin
2       usdt-tether        Tether   USDT     3   False       True  token
3  bnb-binance-coin  Binance Coin    BNB     4   False       True   coin
4        sol-solana        Solana    SOL     5   False       True   coin
2024-09-08 23:55:42,297 - INFO - Number of coins to fetch: 5
2024-09-08 23:55:42,306 - INFO - Number of coins to load: 10
2024-09-08 23:55:42,309 - INFO - First 5 coins to load:                  id          name symbol  rank  is_new  is_active   type
0       btc-bitcoin       Bitcoin    BTC     1   False       True   coin
1      eth-ethereum      Ethereum    ETH     2   False       True   coin
2       usdt-tether        Tether   USDT     3   False    

In [41]:
schema = StructType([
    StructField('id', StringType(), True),
    StructField('name', StringType(), True),
    StructField('symbol', StringType(), True),
    StructField('rank', IntegerType(), True),
    StructField('is_new', BooleanType(), True),
    StructField('is_active', BooleanType(), True),
    StructField('type', StringType(), True),
    StructField('description', StringType(), True),
    StructField('started_at', StringType(), True),  # Initially StringType for conversion later
    StructField('development_status', StringType(), True),
    StructField('hardware_wallet', BooleanType(), True),
    StructField('proof_type', StringType(), True),
    StructField('org_structure', StringType(), True),
    StructField('hash_algorithm', StringType(), True),
    StructField('total_supply', LongType(), True),  # Use LongType for large integers
    StructField('max_supply', LongType(), True),    # Use LongType for large integers
    StructField('last_updated', StringType(), True),  # Initially StringType for conversion later
    StructField('quotes', MapType(StringType(), MapType(StringType(), StringType())), True)  # Extract price from here
])

In [46]:
# Create a DataFrame with the schema enforced
df = session.createDataFrame(check, schema=schema)
# Extract price_usd from quotes['USD']['price']
df = df.withColumn("price_usd", col("quotes.USD.price").cast(DoubleType()))
# Convert 'started_at' and 'last_updated' columns from StringType to TimestampType
df = df.withColumn("started_at", to_timestamp(col("started_at"), "yyyy-MM-dd'T'HH:mm:ss'Z'")) \
       .withColumn("last_updated", to_timestamp(col("last_updated"), "yyyy-MM-dd'T'HH:mm:ss'Z'"))

# Add the new column is_monitored with a default value of True
df = df.withColumn("is_monitored", lit(True))

# Drop the 'quotes' column since it's no longer needed
df = df.drop("quotes")

# Fill any null values with 'NA'
df = df.fillna("NA")

df.show()

+-----------------+------------+--------+----+------+---------+-----+--------------------+-------------------+------------------+---------------+--------------------+--------------------+----------------+------------+----------+-------------------+--------------------+------------+
|               id|        name|  symbol|rank|is_new|is_active| type|         description|         started_at|development_status|hardware_wallet|          proof_type|       org_structure|  hash_algorithm|total_supply|max_supply|       last_updated|           price_usd|is_monitored|
+-----------------+------------+--------+----+------+---------+-----+--------------------+-------------------+------------------+---------------+--------------------+--------------------+----------------+------------+----------+-------------------+--------------------+------------+
|      btc-bitcoin|     Bitcoin|     BTC|   1| false|     true| coin|Bitcoin is a cryp...|2009-01-03 00:00:00|   Working product|           true|      

In [15]:
df_selected = df.select(
    col('id'),
    col('name'),
    col('symbol'),
    col('rank'),
    col('is_new'),
    col('is_active'),
    col('type'),
    col('description'),
    col('started_at'),
    col('development_status'),
    col('hardware_wallet'),
    col('proof_type'),
    col('org_structure'),
    col('hash_algorithm'),
    col('total_supply'),
    col('max_supply'),
    col('last_updated'),
    col('quotes.USD.price').alias('price_usd')  # Extract price_usd from quotes dictionary
)

# Show the resulting DataFrame
df_selected.show(truncate=False)

+-----------------+------------+--------+----+------+---------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+------------------+---------------+-------------------------+--------------------------+----------------+------------+----------+--------------------+

In [20]:
# Replace all null values with "NA"
df = df.fillna("NA")
df_selected.show(truncate=False)

In [39]:
check[0]

{'id': 'btc-bitcoin',
 'name': 'Bitcoin',
 'symbol': 'BTC',
 'rank': 1,
 'is_new': False,
 'is_active': True,
 'type': 'coin',
 'logo': 'https://static.coinpaprika.com/coin/btc-bitcoin/logo.png',
 'tags': [{'id': 'segwit',
   'name': 'Segwit',
   'coin_counter': 12,
   'ico_counter': 0},
  {'id': 'cryptocurrency',
   'name': 'Cryptocurrency',
   'coin_counter': 1149,
   'ico_counter': 40},
  {'id': 'proof-of-work',
   'name': 'Proof Of Work',
   'coin_counter': 996,
   'ico_counter': 0},
  {'id': 'payments',
   'name': 'Payments',
   'coin_counter': 185,
   'ico_counter': 39},
  {'id': 'sha256', 'name': 'Sha256', 'coin_counter': 93, 'ico_counter': 0},
  {'id': 'mining', 'name': 'Mining', 'coin_counter': 576, 'ico_counter': 3},
  {'id': 'lightning-network',
   'name': 'Lightning Network',
   'coin_counter': 8,
   'ico_counter': 0},
  {'id': 'layer-1-l1',
   'name': 'Layer 1 (L1)',
   'coin_counter': 113,
   'ico_counter': 0},
  {'id': 'ftx-holdings',
   'name': 'FTX Holdings',
   'coin_

In [ ]:
session.stop()